In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
import time
from ipywidgets import widgets

df = pd.read_csv('youtube.csv')

In [2]:
# import plotly.io as pio
# pio.renderers.default = "plotly_mimetype+notebook_connected"

In [3]:
template='plotly'
df_gb = df.groupby('name').agg('max').reset_index().sort_values('subscribers (million)', ascending=False)
df_tag_count = df_gb['tag'].value_counts()
colors_map = {i:j for i, j in zip(df_tag_count.index, px.colors.qualitative.Light24[:11])}

# Objective

## Objective {.center style="font-size:0.7em;"}

::: {style="font-size:1em;"}
<br>

- **Data exploration**: Explore some basic descriptive statistics of the data
- **Trends identification**: Identify what kind of channels are popular in the past three years
- **Growth analysis**: Identify the most growing channels, calculated by monthly gained subs
- **Content-creation strategy examination**: Examine what are the possible strategies used by the top growing channels to gain subscribers or viewers
:::

# Data Description

## Data Description {.center style="font-size:0.6em;" }

Data are collected from [SocialBlade](https://socialblade.com/youtube/top/country/id/mostsubscribed) using data scraping technique. It has 11 columns:

- `name`: The channel’s name
- `upload_count`: The number of uploaded content
- `subscribers (million)`: The number of subscribers
- `views`: Total views from all uploaded videos
- `views/video`: Total views divided by upload count
- `tag`: The main category of the channel
- `date_created`: The channel’s creation date
- `age`: Age of the channel (in November 2023)
- `date`: Monthly date indicating when the views and subs were gained
- `views_gained`: Number of views gained each month
- `subs_gained`: Number of subscribers gained each month

I wrote a [full details](/posts/youtube_scraping/youtube_scraping.html) about the data collection process and how I cleaned it.

# Analysis

## Notes {.center style="font-size:0.7em;" }

**Charts are interactive**:

- Click on legend label to exclude it from the chart
- Double click on the legend label to isolate that label
- Hover into a point in the chart to see more details
- Drag or click on the chart to zoom in
- Double click on the chart to zoom out

Let's begin!

## What are the primary categories represented among the top 100 channels? {style="font-size: 0.5em;"}

In [4]:
fig = px.pie(df_gb, values=df_tag_count.values, names=df_tag_count.index, template=template,
            color_discrete_sequence=px.colors.qualitative.Light24)
fig.update_layout(width=1100,
    height=540)
fig.show()

::: {style="font-size:1.25em;"}
<span style="color:#FD3216">Entertainment</span> asserts its dominance with 40 channels, followed by <span style="color:#00FE35">Games</span>, <span style="color:#6A76FC">People</span>, and <span style="color:#FED4C4">Film</span> categories with 14, 11, and 10 channels, respectively. **These four categories alone already represented 75% of the top 100 channels.** 

The rest of the categories demonstrate a smaller but notable presence with 6 or fewer channels each. This highlights the diversity of content on the Indonesian YouTube platform.
:::

## How the amount of subscribers are distributed across the channels? {style="font-size: 0.5em;"}

In [5]:
# to make the legend color consistent with the previous plot
fig = px.bar(df_gb, x='name', y='subscribers (million)', color='tag', template=template,
             hover_data={'rank': df_gb.reset_index().index + 1}, 
             color_discrete_sequence=[colors_map[c] for c in df_gb['tag'].unique()])
# overwrite tick labels    
fig.update_layout(
    barmode = 'stack',
    xaxis = {
        'categoryorder':'array', 
        'categoryarray':df_gb['name'],
        'tickmode': 'array',
        'tickvals': list(range(100)),
        'ticktext': df_gb['name'].str[:15].tolist(),
        'tickangle': -45,
    }
)
fig.update_layout(width=1100,
    height=540)
fig.show()

::: {style="font-size:1.25em;"}
Overall, in addition to its dominance in terms of proportion, the <span style="color:#FD3216">Entertainment</span> category dominates again in terms of subscribers distribution. Although <span style="color:#00FE35">Games</span>, <span style="color:#6A76FC">People</span>, and <span style="color:#FED4C4">Film</span> category appears a lot in the list, they are more evenly distributed than <span style="color:#FD3216">Entertainment</span> category.

Moreover, the distribution looks exponentially decayed. To gain more information, let's plot the cumulative sum.
:::

## How the amount of subscribers are distributed across the channels? {style="font-size: 0.5em;"}

In [6]:
df_gb['cum_subs'] = 100*(df_gb['subscribers (million)']/df_gb['subscribers (million)'].sum()).cumsum()
fig = px.bar(df_gb, x='name', y='cum_subs', color='tag',
             hover_data={'rank': df_gb.reset_index().index + 1}, template=template,
             color_discrete_sequence=[colors_map[c] for c in df_gb['tag'].unique()])
fig.add_vline(x=11.5, line_dash="dot",
              annotation_text="25%", 
              annotation_position="top right")
fig.add_vline(x=32.5, line_dash="dot",
              annotation_text="50%", 
              annotation_position="top right")
fig.update_layout(
    barmode = 'stack',
    xaxis = {
        'categoryorder':'array', 
        'categoryarray':df_gb['name'],
        'tickmode': 'array',
        'tickvals': list(range(100)),
        'ticktext': df_gb['name'].str[:15].tolist(),
        'tickangle': -45,
    }
)
fig.update_layout(width=1100,
    height=540)
fig.show()

::: {style="font-size:1.25em;"}
From this chart, we can infer some information:

- A quarter (25%) of total subscribers are subscribed to the top 12 channels
- Half (50%) of total subscribers are subscribed to the top 33 channels

It reveals insight into the concentration of audience distribution among top channels. It shows **how a minority of channels significantly hold a substantial portion of the overall subscriber.**
:::

## What kind of channels are the most active? {style="font-size: 0.5em;"}

In [7]:
fig = px.treemap(df_gb, path=[px.Constant("all"), 'tag', 'name'], template=template,
                 color='upload_count', values='subscribers (million)',
                 color_continuous_scale='Mint')
fig.update_layout(width=1100,
    height=640)
fig.show()

::: {style="font-size:1.25em;"}
From the chart, we can see that TV channels are the most active content makers. They can upload tens to hundreds thousand of contents (it's 100x more than most channel in this list).

This dominance might stem from their established infrastructure, dedicated production teams, and access to diverse resources. **With the resources, they are able to generate content on a large scale, maintaining a consistent stream of uploads.**
:::

## What kind of channels are the most efficient? {style="font-size: 0.5em;"}

In [8]:
fig = px.treemap(df.drop(df[df['name'] == 'YtCrash'].index),  path=[px.Constant("all"), 'tag', 'name'],
                 color='views/video', values='subscribers (million)',
                 template=template,
                 color_continuous_scale='Mint')
fig.update_layout(width=1100,
    height=640)
fig.show()

::: {style="font-size:1.25em;"}
It appears that the most efficient channels come from kids channel like **Zuni and Family**, **Aishwa Nahla Official**, and **Like Nastya IDN**, that on average gaining more than 10M views per content.

If we compare this chart to the previous one, we can see that **although TV channels are the most active content makers, they are not efficient.** On average, they only gain around 100k views per content.
:::

## When were the channels created? {style="font-size: 0.5em;"}

In [9]:
df_gb['date_created_monthly'] = df_gb['date_created'].values.astype('datetime64[M]')
fig = px.bar(df_gb, x='date_created_monthly', y='subscribers (million)', color='tag', barmode='stack',
             hover_data=['name', 'age'], 
             template=template,
             color_discrete_sequence=[colors_map[c] for c in df_gb['tag'].unique()]
            )
fig.update_traces(width=3000000000)
fig.update_layout(width=1100,
    height=540)
fig.show()

::: {style="font-size:1.25em;"}
Most top channels are created between 2014 and 2020.

Most recent channels are **Yuni Ara** (7.91M subs) and **Klara Tania** (14.5M subs) that were created in 2021.
Oldest channels are **Upin & Ipin** (9.28M subs) and **Raditya Dika** (10.1M subs) that were created in 2007.
:::

## What are the most trending categories? {style="font-size: 0.5em;"}

::: {.panel-tabset}
### by average gained subs

In [10]:
df_gb_ts = df.drop(df[df['name'] == 'GNP Music'].index).groupby(['date', 'tag'])[['subs_gained', 'views_gained']].agg('mean')
df_gb_ts = df_gb_ts.reset_index()
df_gb_ts = df_gb_ts.reset_index()
fig = px.line(df_gb_ts, x="date", y='subs_gained', color='tag', template=template,
             color_discrete_sequence=[colors_map[c] for c in df_gb_ts['tag'].unique()])
fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y")
fig.update_layout(width=1100,
    height=540)
fig.show()

::: {style="font-size:1.25em;"}
On average, each category maintains a consistent rate of subscriber growth every month.
However, there are three categories that stand out from the rest: <span style="color:#6A76FC">People</span>, <span style="color:#FED4C4">Film</span>, and <span style="color:#FD3216">Entertainment</span>. These categories are gaining more subscribers in recent months compared to their previous months' gains.
:::

### by average gained views

In [11]:
fig = px.line(df_gb_ts, x="date", y='views_gained', color='tag', template=template,
             color_discrete_sequence=[colors_map[c] for c in df_gb_ts['tag'].unique()])
fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y")
fig.update_layout(width=1100,
    height=540)
fig.show()

::: {style="font-size:1.25em;"}
In general, monthly gained views at each category are unpredictable and tend to fluctuate. However, <span style="color:#0DF9FF">News</span> and <span style="color:#F6F926">Education</span> category are the only ones that consistently gained around 100M viewers every month.

Also if you notice, there are several months where gained views are negative in that month. This can be happened if some videos are taken down by Youtube, or it's just deleted by the channel. This leads to a 'negative' net views from the previous month.
:::

:::

## What are the most growing channels? (by gained subs) {style="font-size: 0.5em;"}


In [12]:
# get top five growing channel
top_five = df.groupby('name')['subs_gained'].agg('sum').sort_values(ascending=False).head(5)
top_five = df[df['name'].isin(top_five.index)]

fig = px.line(top_five, x="date", y='subs_gained', color='name', template=template,
              hover_data='tag')
fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y")
fig.update_layout(width=1100,
    height=540)
fig.show()

::: {style="font-size:1.25em;"}
Five most growing channel (in terms of gained subs) in the last three years are presented on the chart. Among the five, two come from <span style="color:#FD3216">Entertainment</span>, two from <span style="color:#6A76FC">People</span>, and one from <span style="color:#00FE35">Games</span>. For the newcomers (**Willie Salim** and **Vilmei**), their rapid growth started from early 2022.
Conversely, the remaining three channels began their growth only in the recent months of 2023.
:::

## What are the most growing channels? (by gained subs) {.scrollable style="font-size: 0.5em;"}

In [13]:
#df.groupby('name')['subs_gained'].agg('sum').sort_values(ascending=False).head(5)
#unsorted output: [22000000, 16100000, 18100000, 23042000, 14457700]
top_five = df_gb[df_gb['name'].isin(['Willie Salim', 'Jess No Limit', 'Frost Diamond', 'Ricis Official',
       'Vilmei'])][['name', 'subscribers (million)']]
top_five['subs'] = top_five['subscribers (million)']*1000000

top_five['subs initial'] = top_five['subs'] - [22000000, 16100000, 18100000, 23042000, 14457700]

fig = px.bar(top_five, x='name', y=['subs', 'subs initial'],
      barmode='group', template=template,)
fig.update_layout(width=1100,
    height=540)
fig.show()

::: {style="font-size:1.25em;"}
Not only Willie Salim and Vilmei gained many subscribers in the recent three years: They did so with **zero**^[The bar chart overestimates the initial value because it's calculated from the difference between all time total subs and total subs from the past 3 years. However, the data for November monthly subs aren't available yet. So the value on the bar chart includes the partial subs gained from November 2023.] subscribers at the start! While impressive, it's not entirely unexpected because they already have many followers on their TikTok account.
:::
::: {.smaller}
notes
:::
::: aside

:::

## What are the most growing channels in each category? {style="font-size: 0.5em;"}

In [14]:
# get top five growing channel
def show_top_five(tag='Entertainment'):
    top_five = df[df['tag'] == tag].groupby('name')['subs_gained'].agg('sum').sort_values(ascending=False).head(5)
    top_five = df[df['name'].isin(top_five.index)]

    fig = px.line(top_five, x="date", y='subs_gained', color='name', template=template,
                  hover_data='tag')
    fig.update_xaxes(
        dtick="M1",
        tickformat="%b\n%Y")
    if tag in ['Education', 'Comedy', 'Animals']:
        fig.update_layout(width=1100,
             height=540,
             legend=dict(
             yanchor="top",
             y=0.99,
             xanchor="right",
             x=0.99)
        )
    else:
        fig.update_layout(width=1100,
             height=540,
             legend=dict(
             yanchor="top",
             y=0.99,
             xanchor="left",
             x=0.01)
        )
    fig.show()

::: {.panel-tabset}

### Entertainment

In [15]:
# get top five growing channel
show_top_five(tag='Entertainment')

### Games

In [16]:
show_top_five(tag='Games')

### People

In [17]:
show_top_five(tag='People')

### Education

In [18]:
show_top_five(tag='Education')

### Comedy

In [19]:
show_top_five(tag='Comedy')

### News

In [20]:
show_top_five(tag='News')

### Howto

In [21]:
show_top_five(tag='Howto')

### Film

In [22]:
show_top_five(tag='Film')

### Music

In [23]:
show_top_five(tag='Music')

### Tech

In [24]:
show_top_five(tag='Tech')

### Animals

In [25]:
show_top_five(tag='Animals')

:::

## What are the most growing channels? (by gained views) {style="font-size: 0.5em;"}

In [26]:
# get top five growing channel

top_five = df.groupby('name')['views_gained'].agg('sum').sort_values(ascending=False).head(5)
top_five = df[df['name'].isin(top_five.index)]

fig = px.line(top_five, x="date", y='views_gained', color='name',
              hover_data='tag', template=template,)
fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y")
fig.update_layout(width=1100,
    height=540)
fig.show()

::: {style="font-size:1.25em;"}
We can see that there are two <span style="color:#0DF9FF">News</span> channels on the chart. We know they will be on the list because they upload at a very high frequency. Let's remove them so we can get the actual youtubers who get most viewers in the past three years.
:::

## What are the most growing channels? (by gained views) {style="font-size: 0.5em;"}

In [27]:
# get top five growing channel
TV = ['TRANS7 OFFICIAL', 'KOMPASTV', 'tvOneNews', 'Tribunnews', 'Tribun Timur', 'RCTI - LAYAR DRAMA INDONESIA', 'SCTV',
      'TRANS TV Official', 'CNN Indonesia', 'MNCTV OFFICIAL', 'Netmediatama', 'Indosiar', 'MD Entertainment']
top_five = df[~df['name'].isin(TV)].groupby('name')['views_gained'].agg(
    'sum').sort_values(ascending=False).head(5)
top_five = df[df['name'].isin(top_five.index)]

fig = px.line(top_five, x="date", y='views_gained', color='name', template=template,
              hover_data='tag')
fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y")
fig.update_layout(width=1100,
                  height=540,
                  legend=dict(
                      yanchor="top",
                      y=1.03,
                      xanchor="right",
                      x=0.99)
                  )
fig.show()

::: {style="font-size:1.25em;"}
After removed <span style="color:#0DF9FF">News</span> channels, five most growing channel (in terms of gained views) in the last three years are presented on the chart. Among the five, two come from <span style="color:#FED4C4">Film</span>, one from <span style="color:#F6F926">Education</span>, one from <span style="color:#6A76FC">People</span>, and one from <span style="color:#FE00CE">Comedy</span>. Interestingly, the two <span style="color:#FED4C4">Film</span> channels are newcomers. They rapidly gaining views right after they started making content in mid 2021.
:::

## What are the most growing channels? (by gained views) {.scrollable style="font-size: 0.5em;"}

In [28]:
# df[~df['name'].isin(TV)].groupby('name')['views_gained'].agg('sum').sort_values(ascending=False).head(5)
# unsorted output: [6852681678, 8979074153, 7996975129, 5894562311, 8165543528]
top_five = df_gb[df_gb['name'].isin(['Klara Tania', 'Jeevan Akawa', 'Mikael TubeHD', 'BabyBus - Cerita & Lagu Anak-anak',
                                     'Ale Khin'])][['name', 'views']]

top_five['views initial'] = top_five['views'] - [6852681678, 8979074153, 7996975129, 5894562311, 8165543528]

fig = px.bar(top_five, x='name', y=['views', 'views initial'], template=template,
      barmode='group')
fig.update_layout(width=1100,
    height=540)
fig.show()

::: {style="font-size:1.25em;"}
We see the similar pattern as before, where two of the most growing channels were starting from zero^[The bar chart overestimates the initial value because it's calculated from the difference between all time total views and total views from the past 3 years. However, the data for November monthly views aren't available yet. So the value on the bar chart includes the partial views gained from November 2023.]! The difference here is that **Klara Tania** and **Ale Khin** had smaller fanbase on TikTok compared to **Willie Salim** and **Vilmei**.
:::
::: {.smaller}
notes
:::
::: aside
:::

## Are the most efficient channels still popular? {.scrollable style="font-size: 0.5em;"}

:::{.panel-tabset}

### By subs

In [29]:
# get top five growing channel

top_five = df.groupby('name')['views/video'].agg('sum').sort_values(ascending=False).head(5)
top_five = df[df['name'].isin(top_five.index)]

fig = px.line(top_five, x="date", y='subs_gained', color='name',
              hover_data='tag', template=template,)
fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y")
fig.update_layout(width=1100,
             height=540,
             legend=dict(
             yanchor="top",
             y=0.99,
             xanchor="right",
             x=0.8)
        )
fig.show()

::: {style="font-size:1.25em;"}
While these channels' efficiency, measured in views per upload, has traditionally been high, there has been a gradual decline in their popularity in the last three years.
:::

### By views

In [30]:
# get top five growing channel

top_five = df.groupby('name')['views/video'].agg('sum').sort_values(ascending=False).head(5)
top_five = df[df['name'].isin(top_five.index)]

fig = px.line(top_five, x="date", y='views_gained', color='name',
              hover_data='tag', template=template,)
fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y")
fig.update_layout(width=1100,
             height=540,
             legend=dict(
             yanchor="top",
             y=0.99,
             xanchor="right",
             x=0.99)
        )
fig.show()

::: {style="font-size:1.25em;"}
While these channels' efficiency, measured in views per upload, has traditionally been high, there has been a gradual decline in their popularity in the last three years.
:::

:::

## What is the relationship between each pair of numeric variables? {.scrollable style="font-size: 0.5em;"}

In [31]:
fig = px.scatter_matrix(df,
    dimensions=["upload_count", "views", "subscribers (million)", "age", "views/video"],
    color="tag",
    color_discrete_sequence=[colors_map[c] for c in df_gb['tag'].unique()],
    hover_data=['name'], template=template,
    labels={col:col.replace('_', ' ') for col in df.columns}) # remove underscore
fig.update_traces(diagonal_visible=False)
fig.update_layout(width=1100,
    height=650)
fig.show()

::: {style="font-size:1.25em;"}
It doesn't seems there's any correlated variables here, except for subscribers and views variables that somehow seems correlated. However, we can see that there are many outliers here. If you hover on the outliers, you will see that many of them are TV channels. These channels don't represent a typical youtubers, so it's safe to get rid of them from this chart. We will also get rid of **Zuni and Family** because they behave as an outlier too.
:::

## What is the relationship between each pair of numeric variables? {.scrollable style="font-size: 0.5em;"}

In [32]:
df_no_outlier = df[(~df['name'].isin(TV)) & (df['name'] != "Zuni and Family")]
fig = px.scatter_matrix(df_no_outlier,
    dimensions=["upload_count", "views", "subscribers (million)", "age", "views/video"],
    color="tag", template=template,
    color_discrete_sequence=[colors_map[c] for c in df_no_outlier['tag'].unique()],
    hover_data=['name'],
    labels={col:col.replace('_', ' ') for col in df.columns}) # remove underscore
fig.update_traces(diagonal_visible=False)
fig.update_layout(width=1100,
    height=650)
fig.show()

::: {style="font-size:1.25em;"}
Now we can see that:

- Efficiency and views are positively correlated
- Efficiency and upload_count are negatively correlated
- Views and subscribers are positively correlated

The first two points are kinda obvious as we use views to obtain efficiency. The last point tells us that the most subscribed channels are having the most views.
:::

## Correlation coefficient between each pair of numeric variables {.scrollable style="font-size: 0.5em;"}

In [33]:
#df_corr = df[df.select_dtypes(include='number').columns.union(['name'])][~df['name'].isin(TV)].groupby('name').agg('sum').corr().round(3)
df_corr = df_no_outlier[['name', 'upload_count', 'views', 'subscribers (million)',
                                    'age', 'views/video']].groupby('name').agg('sum').corr().round(3)
mask = np.triu(np.ones_like(df_corr, dtype=bool))
df_mask = df_corr.mask(mask)

fig = ff.create_annotated_heatmap(z=df_mask.to_numpy(),
                                  x=df_mask.columns.tolist(),
                                  y=df_mask.columns.tolist(),
                                  colorscale=px.colors.diverging.RdBu,
                                  hoverinfo="none",  # Shows hoverinfo for null values
                                  showscale=True, ygap=1, xgap=1,
                                  zmin=-1, zmax=1
                                  )

fig.update_xaxes(side="bottom")

fig.update_layout(
    title_text='Heatmap',
    title_x=0.5,
    width=1100,
    height=540,
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    yaxis_autorange='reversed',
    template='plotly_white'
)

# NaN values are not handled automatically and are displayed in the figure
# So we need to get rid of the text manually
for i in range(len(fig.layout.annotations)):
    if fig.layout.annotations[i].text == 'nan':
        fig.layout.annotations[i].text = ""

fig.show()

::: {style="font-size:1.25em;"}
Here we can see that:

- Efficiency and views are positively correlated
- Efficiency and upload_count are negatively correlated
- Views and subscribers are positively correlated

The first two points are kinda obvious as we use views to obtain efficiency. The last point tells us that the most subscribed channels are having the most views.
:::

# Takeaways

## Youtube Shorts {style="font-size:0.7em; margin-top: 40px"}

::: {style="font-size:1em;"}
<br>
In the last three years, particularly in Indonesia, Youtube Shorts has been widely used to boost audience engagement and channel growth. In our analysis (especially [this](#what-are-the-most-growing-channels-by-gained-subs) and [this](#what-are-the-most-growing-channels-by-gained-views-1) chart), we have seen that **Shorts** can be utilized effectively by everyone.

- Jess No Limit recently gained more than 20 million subscribers in 2023 due to his intensity in uploading **Shorts** content since April 2023.
- Ricis, who previously only uploaded several Shorts contents about her family, gained more subscribers after she creates more **Shorts** contents about general topic (not only her family).
- Although Frost Diamond has always been uploading youtube videos consistently, he gained massive amount of subscriber only after he started to actively creating **Shorts** content.
- Klara Tania and Ale Khin gained billions of viewers from making **Shorts** contents despite they only started their Youtube career in mid 2021 and have little popularity before it.
:::

## TikTok stars go Tube {style="font-size:0.7em; margin-top: 40px"}

::: {style="font-size:1em;"}
<br>
Thanks to the success of YouTube Shorts, TikTok stars like Willie Salim and Vilmie are rapidly growing their presence on YouTube. Both have achieved remarkable success within three years.

- Willie Salim gained more than 23M subscribers and joined in top 10 most subscribed Indonesian youtubers. Moreover, he achieved that with less than one thousand uploaded contents, which is very low.
- Vilmei gained more than 14M subscribers and joined in top 30 most subscribed Indonesian youtubers.

:::

## Kids contents are not that popular anymore {style="font-size:0.7em; margin-top: 40px"}

::: {style="font-size:1em;"}
<br>
Kids' YouTube channels have demonstrated exceptional efficiency in terms of views per upload count. However, there has been a slight decrease in their popularity in Indonesia over recent years, as we have seen [here](#are-the-most-efficient-channels-still-popular).
:::

## It is never too late {style="font-size:0.7em; margin-top: 40px"}

::: {style="font-size:1em;"}
<br>
Throughout its history, Youtube has always been a dynamic platform for content creators. If you're considering diving into content creation, there's no better time than now. Notably, creators like Yuni Ara, Klara Tania, and Ale Khin were just started they journey in recent years. Their success emphasizes that choosing the right niche with the right strategy will raise the potential to boost your popularity rapidly.
:::

# Suggestion

## Suggestion {style="font-size:0.7em; margin-top: 40px"}

::: {style="font-size:1em;"}
<br>

- I suggest to **create more Shorts contents**, especially for big youtubers, as many of them haven't utilized this potential feature. They could easily gain millions of subscribers by shifting into producing Shorts contents.
- For tiktokers, I highly suggest to **expand their presence in youtube through youtube shorts**. They can gain more audience by uploading the same contents in tiktok and youtube.
- For the aspiring creators, **it's never too late to start the journey** in the content creation industry. Some of the current top youtubers only started their career in the past three years.
:::